In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

x_train_cnn = np.expand_dims(x_train, axis=-1)
x_test_cnn  = np.expand_dims(x_test, axis=-1)

print("MLP input:", x_train.shape)
print("CNN input:", x_train_cnn.shape)


In [ ]:
mlp = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax"),
])

mlp.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

mlp.summary()


In [ ]:
cnn = keras.Sequential([
    layers.Conv2D(16, kernel_size=3, activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=2),
    layers.Conv2D(32, kernel_size=3, activation="relu"),
    layers.MaxPooling2D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax"),
])

cnn.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

cnn.summary()


In [ ]:
EPOCHS = 5
BATCH = 64

hist_mlp = mlp.fit(
    x_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH,
    validation_split=0.1,
    verbose=1
)

hist_cnn = cnn.fit(
    x_train_cnn, y_train,
    epochs=EPOCHS,
    batch_size=BATCH,
    validation_split=0.1,
    verbose=1
)


In [ ]:
mlp_loss, mlp_acc = mlp.evaluate(x_test, y_test, verbose=0)
cnn_loss, cnn_acc = cnn.evaluate(x_test_cnn, y_test, verbose=0)

print("MLP  -> loss:", float(mlp_loss), "acc:", float(mlp_acc))
print("CNN  -> loss:", float(cnn_loss), "acc:", float(cnn_acc))


In [ ]:
def trainable_params(model):
    return int(np.sum([np.prod(v.shape) for v in model.trainable_weights]))

mlp_params = trainable_params(mlp)
cnn_params = trainable_params(cnn)

print("MLP trainable params:", mlp_params)
print("CNN trainable params:", cnn_params)


In [ ]:
os.makedirs("models", exist_ok=True)

mlp_path = os.path.join("models", "mlp_model.h5")
cnn_path = os.path.join("models", "cnn_model.h5")

mlp.save(mlp_path, include_optimizer=True)
cnn.save(cnn_path, include_optimizer=True)

mlp_size_mb = os.path.getsize(mlp_path) / (1024 * 1024)
cnn_size_mb = os.path.getsize(cnn_path) / (1024 * 1024)

print("MLP saved size (MB):", mlp_size_mb)
print("CNN saved size (MB):", cnn_size_mb)


In [ ]:
def estimate_training_memory_bytes(trainable_param_count, dtype_bytes=4, optimizer="adam"):
    param_bytes = trainable_param_count * dtype_bytes
    if optimizer.lower() == "adam":
        opt_state_bytes = 2 * param_bytes
    else:
        opt_state_bytes = 0
    grad_bytes = param_bytes
    total = param_bytes + opt_state_bytes + grad_bytes
    return total

mlp_mem_mb = estimate_training_memory_bytes(mlp_params) / (1024 * 1024)
cnn_mem_mb = estimate_training_memory_bytes(cnn_params) / (1024 * 1024)

print("Estimated training memory (MB) - MLP:", mlp_mem_mb)
print("Estimated training memory (MB) - CNN:", cnn_mem_mb)


In [ ]:
def infer_flops(model, input_shape):
    try:
        concrete = tf.function(model).get_concrete_function(
            tf.TensorSpec([1] + list(input_shape), model.inputs[0].dtype)
        )
        frozen_func, graph_def = tf.python.framework.convert_to_constants.convert_variables_to_constants_v2_as_graph(concrete)
        with tf.Graph().as_default() as graph:
            tf.graph_util.import_graph_def(graph_def, name="")
            run_meta = tf.compat.v1.RunMetadata()
            opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
            flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd="op", options=opts)
        return int(flops.total_float_ops) if flops is not None else None
    except Exception as e:
        print("FLOPs profiling failed:", e)
        return None

mlp_flops_inf = infer_flops(mlp, (28, 28))
cnn_flops_inf = infer_flops(cnn, (28, 28, 1))

mlp_flops_train = (2 * mlp_flops_inf) if mlp_flops_inf is not None else None
cnn_flops_train = (2 * cnn_flops_inf) if cnn_flops_inf is not None else None

print("MLP FLOPs inference:", mlp_flops_inf)
print("MLP FLOPs training :", mlp_flops_train)
print("CNN FLOPs inference:", cnn_flops_inf)
print("CNN FLOPs training :", cnn_flops_train)


In [ ]:
import pandas as pd

summary = pd.DataFrame([
    {
        "Model": "MLP",
        "Test Accuracy": float(mlp_acc),
        "Test Loss": float(mlp_loss),
        "Trainable Parameters": int(mlp_params),
        "Saved Model Size (MB)": float(mlp_size_mb),
        "FLOPs (Training)": mlp_flops_train,
        "FLOPs (Inference)": mlp_flops_inf,
        "Training Memory (MB)": float(mlp_mem_mb),
    },
    {
        "Model": "CNN",
        "Test Accuracy": float(cnn_acc),
        "Test Loss": float(cnn_loss),
        "Trainable Parameters": int(cnn_params),
        "Saved Model Size (MB)": float(cnn_size_mb),
        "FLOPs (Training)": cnn_flops_train,
        "FLOPs (Inference)": cnn_flops_inf,
        "Training Memory (MB)": float(cnn_mem_mb),
    },
])

summary
